In [ ]:
import shap
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#prepare data
df=gpd.read_file('Data/NepalEqUSGS.shp')
constcols=['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGV_Usgs']
Xdata=df[constcols].to_numpy()
Ydata=df.Landslide.to_numpy()
max=Xdata.max(axis=0)
min=Xdata.min(axis=0)
norm_xdata=(Xdata-min)/(max-min)

In [ ]:
model=tf.keras.models.load_model('USGS_Eq_Clf_model.h5')

In [ ]:
explainer = shap.DeepExplainer(model, norm_xdata[np.random.choice(norm_xdata.shape[0], 100, replace=False)])

shap_values = explainer.shap_values(np.array(norm_xdata,dtype=np.float32))

In [ ]:
shap.summary_plot(shap_values[0], plot_type = 'violin', feature_names = constcols,show=False)
plt.savefig('Exaiplots/overallimpactDistribution.pdf')

In [ ]:
shap.initjs()
shap.force_plot(0.5, shap_values[0][8], features = constcols,matplotlib=True,feature_names = constcols,show=False)
plt.savefig('Exaiplots/singlePositiveExample.pdf')

In [ ]:
shap.initjs()
shap.force_plot(0.5, shap_values[0][10], features = constcols,matplotlib=True,feature_names = constcols,show=False)
plt.savefig('Exaiplots/SingleNegativeExample.pdf')

In [ ]:
shap.decision_plot(explainer.expected_value[0], shap_values[0], features = norm_xdata, feature_names = constcols,show=False,ignore_warnings=True)
plt.savefig('Exaiplots/DecisionPlot_all.pdf')

In [ ]:
import matplotlib.pyplot as plt
shap.plots._waterfall.waterfall_legacy(explainer.expected_value[0], shap_values[0][10], feature_names = constcols)
plt.savefig('Exaiplots/additiveNature_Negative.pdf')

In [ ]:
import matplotlib.pyplot as plt
shap.decision_plot(explainer.expected_value[0], shap_values[0][10], feature_names = constcols,show=False)
plt.savefig('waterfall_chk.png',dpi=300)

In [ ]:
for i in range(len(shap_values[0])):
    shap.decision_plot(explainer.expected_value[0], shap_values[0][i], feature_names = constcols,show=False)
    plt.savefig(f'/home/dahala/Documents/ashokdahal.github.io/plotapp/{str(i)}.png',dpi=300)
    plt.tight_layout()
    plt.clf()

In [ ]:
shap.summary_plot(shap_values[0], norm_xdata,feature_names = constcols,show=False)
plt.colorbar()
plt.tight_layout()
plt.savefig('Exaiplots/Norm_value_Impact_on_SHAP.pdf')


In [ ]:
shap.group_difference_plot(shap_values[0],group_mask=Ydata.astype(bool),feature_names=constcols,show=False)
plt.savefig('Exaiplots/Mean_SHAP_difference_between_landslide_noLandslide.pdf')

In [ ]:
fig,axs=plt.subplots(2, 4, figsize=(16,8),sharex=False,sharey=False)
i=0
j=0
for k in range(8):
    plt.axes(axs[i,j])
    shap.embedding_plot(k,shap_values[0],feature_names=constcols,show=False)
    j+=1
    if j==4:
        i=1
        j=0
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
fig,axs=plt.subplots(2, 4, figsize=(16,8),sharex=True,sharey=True)
df_sub=df[constcols]
i=0
j=0
for col in range(8):
    shap.dependence_plot((col), shap_values[0], norm_xdata,interaction_index=None,show=False,cmap='viridis',ax=axs[i,j],feature_names=constcols)
    j+=1
    if j ==4:
        i=1
        j=0
    #axs[i,j].colorbar()
plt.tight_layout()
plt.savefig('Exaiplots/Variable_Influence_Distribution.pdf')

In [ ]:
preds=model.predict(norm_xdata)

In [ ]:
for i in range(len(df)):
    df.loc[i, 'url'] = f"https://raw.githubusercontent.com/ashokdahal/ashokdahal.github.io/master/plotapp/{i}.png"
    df.loc[i,'susceptibility']=preds[i][0]

In [ ]:
df_web=df[['url','susceptibility','geometry']]

In [ ]:
df_web.to_file('Exaiplots/WebappData.gpkg',driver='GPKG')